In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Linear Regression

Question: Write code to implement linear regression from scratch without using
any machine learning libraries like scikit-learn.

Answer Guidance: Be prepared to:
* Handle data input and preprocessing.
* Compute the OLS estimates manually.
* Implement matrix operations using libraries like NumPy.
* Optionally, write functions to predict new data points and calculate metrics 
 like Mean Squared Error (MSE).


In [2]:
# Generalizing for Multiple Linear Regression

def multiple_linear_regression(X,y):

    """
    Estimates the coefficients for a multiple linear regression model using the normal equation.

    This function computes the coefficients (weights) for a multiple linear regression model
    by solving the normal equation: beta = (X^T * X)^(-1) * X^T * y. It works for any number
    of predictors (independent variables) and includes an intercept term.

    Parameters:
    ----------
    X : array-like or list of lists
        Feature matrix containing the independent variables.
        Shape: (n_samples, n_features)
    
    y : array-like or list
        Target vector containing the dependent variable.
        Shape: (n_samples,)

    Returns:
    -------
    beta : numpy.ndarray
        Coefficient vector including the intercept term and coefficients for each predictor.
        Shape: (n_features + 1,)
    """    
    
    # Conver inout to numpy (in case they are dataframes..)
    X = np.array(X)
    y = np.array(y)

    # Add intercept
    ones = np.ones((X.shape[0], 1))  # vector of ones
    X = np.concatenate((ones, X), axis =1)

    #Compute the elements of the normal equations
    #     A = (X'X)**(-1) 
    #     B = X'Y
    #  beta = A* B is then the coefficient vector

    A = np.linalg.inv(( X.T.dot(X)))
    B = X.T.dot(y)

    beta = A.dot(B)

    return(beta)

Note: In NumPy, the * operator performs element-wise multiplication, not matrix multiplication.
To perform matrix multiplication, use the .dot() method or the @ operator.


`A.dot(B)`:  Correct way to perform matrix multiplication in NumPy

In [3]:
def predict(X_new, beta):
    
    # Convert input to numpy array
    X_new = np.array(X_new)

    # Add intercept term
    ones = np.ones((X_new.shape[0]),1 )
    X_new = np.concat((ones, X_new), axis = 1)

    # Calculate prediction 

    y_pred = X_new.dot(beta)

    return y_pred

def mean_squared_error(y_true, y_pred):
    mse = np.mean((y_true - y_pred)**2)
    return mse

A possible follow up question would be **How would you implement linear 
regression using gradient descent instead of the normal equation?** 

This gradient approach is especially useful when dealing with a large number of 
features or when the matrix inversion in the normal equation becomes computationally 
expensive. Gradient descent works by iteratively adjusting the coefficients to minimize the cost function, typically the Mean Squared Error (MSE) for linear regression.

### Key Concepts of Gradient Descent for Linear Regression

#### 3. Cost Function
The cost function measures how well the model's predictions match the actual data. For linear regression, the cost function is the Mean Squared Error (MSE):

$$
J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right)^2
$$

where:
- $ m $ is the number of training examples.
- $ h_{\theta}(x^{(i)}) $ is the predicted value for the $ i $-th sample.
- $ y^{(i)} $ is the actual value for the $ i $-th sample.

The goal of gradient descent is to find the values of $ \theta $ that minimize $ J(\theta) \).

#### 4. Gradient Descent Algorithm
Gradient Descent is an iterative optimization algorithm that updates the coefficients in the direction of the negative gradient of the cost function:

$$
\theta_j = \theta_j - \alpha \frac{\partial J(\theta)}{\partial \theta_j}
$$

where:
- $ \theta_j $ is the $ j $-th parameter.
- $ \alpha $ is the learning rate, controlling the step size.
- $ \frac{\partial J(\theta)}{\partial \theta_j} $ is the partial derivative of the cost function with respect to $ \theta_j $.

#### 5. Gradient Calculation
For linear regression, the gradient of the cost function with respect to each parameter $ \theta_j $ is calculated as follows:

$$
\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
$$

where:
- $ x_j^{(i)} $ is the value of the $ j $-th feature for the $ i $-th sample.

#### 6. Update Rule
The update rule for each coefficient in the gradient descent algorithm is:

$$
\theta_j = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
$$

This update rule is applied iteratively for all coefficients until convergence.

#### 7. Learning Rate
The learning rate $ \alpha $ controls the size of the steps taken towards the minimum of the cost function. Choosing the right learning rate is crucial:
- **Too small:** The algorithm will take too long to converge.
- **Too large:** The algorithm may overshoot the minimum or even diverge.

#### 8. Convergence
The gradient descent algorithm converges when the cost function $ J(\theta) $ reaches its minimum value. This can be monitored by checking the change in cost over iterations or by setting a maximum number of iterations.

#### 9. Variants of Gradient Descent
- **Batch Gradient Descent:** Uses all training examples to compute the gradient.
- **Stochastic Gradient Descent (SGD):** Uses one training example at a time to compute the gradient, which can be faster but introduces more noise in the updates.
- **Mini-batch Gradient Descent:** Uses a small subset (mini-batch) of the training examples to compute the gradient, balancing the efficiency and stability of the updates.

#### 10. Summary
Gradient Descent is a powerful algorithm for optimizing linear regression models, especially when dealing with large datasets. By iteratively updating the coefficients in the direction of the negative gradient of the cost function, it finds the optimal parameters that minimize the error between the predicted and actual values.

In [4]:
# So let's write these beast

def linear_regression_gradient_descent(X, y, alpha=0.01, n_iterations=1000):
    # alpha is the learning rate`
    # Convert input to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Number of training examples and features
    m = X.shape[0]  # Number of samples
    n = X.shape[1]  # Number of features

    # Add intercept
    ones = np.ones((m, 1))  # vector of ones
    X = np.concatenate((ones, X), axis =1)

    # Initialize theta (coefficient vector) with zeros
    beta = np.zeros(n+1)

    # Initialize cost function (optional)
    cost_history = []

    ###########################################################################
    # Gradient Descent

    for iter in range(n_iterations):

        y_pred = X.dot(beta)               # Calculate predictions
        error = y_pred - y                  # Calculate errors
        gradients = (1/m) *  np.dot(X.T, error) # Calculate gradients
        beta = beta - alpha * gradients   # Update rule

        cost_history.append( (1/(2*m)) * np.sum(error**2)) # Optional: update cost_history 

    return beta, cost_history

# Logistic Regression.

The logistic regression model arises from the desire to model the posterior 
probabilities of the K classes via linear functions in x, while at the same time
ensuring that they sum to one and remain in [0,1].

The **sigmoid function** is maps any real-valued number into a value between 0 
and 1, making it suitable for modeling probabilities:

$$   \sigma(z) = \frac{1}{1+ e^(-z)}  $$

where $ z = \beta_0 + \beta X$




In [5]:
def logistic_regression(X, y, alpha=0.01, n_iterations=1000):
    # alpha is the learning rate`
    # Convert input to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Number of training examples and features
    m = X.shape[0]  # Number of samples
    n = X.shape[1]  # Number of features

    # Add intercept
    ones = np.ones((m, 1))  # vector of ones
    X = np.concatenate((ones, X), axis =1)

    # Initialize theta (coefficient vector) with zeros
    beta = np.zeros(n+1)

    ###########################################################################
    # Gradient Descent
    ###########################################################################

    for iter in range(n_iterations):
        
        z = X.dot(beta)
        y_pred = 1/(1 + np.exp(-z))              # Calculate predictions
        error = y_pred - y                  # Calculate errors
        gradients = (1/m) *  np.dot(X.T,error) # Calculate gradients
        beta = beta - alpha * gradients   # Update rule

    return beta

In [6]:
import numpy as np
class LinearModels:
    
    def __init__(self):
        self.beta = None
        self.n = None

    def sigmoid(self, X):
        z = X.dot(self.beta)
        prob = 1/(1 + np.exp(-z))
        return prob

    def fit(self, X_train, y_train, type = 'linear', alpha=0.01,
                    min_cost = 1e-10,  max_iter=1000):

        """
        The implementations based on gradient descent for linear and logistic 
        regression are almost identical. The only difference is the computation of 
        y_pred. This code makes this clear, generalizing the linear regression 
        function to include a new parameter, 'type':

        Parameters:
        ----------
        X : array-like
            Feature matrix, shape (n_samples, n_features).
        
        y : array-like
            Target vector, shape (n_samples,).
        
        type : string, optional
            Either 'linear' or 'logistic'. Defines the type of regression performed.
            Default is 'linear'.
        
        alpha : float, optional
            Learning rate for gradient descent. Default is 0.01.
        
        max_iter : int, optional
            Number of iterations for gradient descent. Default is 1000.

        Returns:
        -------
        beta : numpy.ndarray
            Coefficient vector, shape (n_features + 1,).
        """
        # alpha is the learning rate`
        # Convert input to numpy arrays
        X = np.array(X_train)
        y = np.array(y_train)

        # Number of training examples and features
        m = X.shape[0]  # Number of samples 
        self.n = X.shape[1]  # Number of features (without intercept)

        # Add intercept
        ones = np.ones((m, 1))  # vector of ones
        X = np.concatenate((ones, X), axis =1)

        # Initialize beta (coefficient vector) with zeros
        self.beta = np.zeros(self.n+1)

        # Initialize cost function (optional)
        cost =np.inf
        iter = 0
        ###########################################################################
        # Gradient Descent
        ###########################################################################

        while iter <= max_iter and cost > min_cost:
            iter +=1
            if type == 'linear':
                y_pred = X.dot(self.beta)  
            elif type == 'logistic':
                y_pred = self.sigmoid(X) # this is actually the probability of y = 1
            else:
                raise ValueError('Type must be either linear or logistic') 

            # Calculate predictions
            error = y_pred - y                  # Calculate errors
            gradients = (1/m) *  np.dot(X.T,error) # Calculate gradients
            self.beta = self.beta - alpha * gradients   # Update rule

            if type == 'linear':
                # For linear regression, the cost function is Mean Squared Error (MSE)
                cost = (1/(2*m)) * np.sum(error**2) 
            else:
                # For logistic regression, the cost function is the negative log-likelihood
                cost = (-1 / m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
            
        return self.beta

    def predict(self, X_test, type = 'linear', prob_threshold = 0.5):
        
        X = np.array(X_test)
        if self.n != X.shape[1]:
            raise ValueError('The number of features in the test data does not correspond to the number of regressors in the training data.')
        m = X.shape[0]  # Number of samples 
        ones = np.ones((m, 1)) 
        

        if type == 'linear':

           X = np.concatenate((ones, X), axis =1)
           y_pred = X.dot(self.beta) 
                
        elif type == 'logistic':
            X = np.concatenate((ones, X), axis =1)
            prob = self.sigmoid(X)
            y_pred = np.where(prob > prob_threshold, 1, 0)

        return y_pred

### Additional notes on logistic regression:


Question: How would you modify your logistic regression implementation to 
include L2 regularization?

We can add a regularization term to the cost function to penalize large weights.

    - write deep-dive.

Question: Which metrics would you use to evaluate your logistic regression model,
and how would you implement them?

Discus accuracy metrics:
	•	Precision
	•	Recall
	•	F1 Score
	•	Confusion Matrix
	•	ROC Curve and AUC

To-do: write code


What are the key assumptions of logistic regression, and how can you test them 
with your data?

List assumptions such as:
	•	Linearity of independent variables and log-odds.
	•	Independence of errors.
	•	Lack of multicollinearity.
	•	Large sample size.

    

$$ J(\boldsymbol{\beta}) = - \frac{1}{m} \sum_{i=1}^{m} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}i) \right] + \frac{\lambda}{2m} \sum{j=1}^{n} \beta_j^2 $$


Final comaprison/resume of logistic and linear regressions.


| **Aspect**                     | **Linear Regression**                                                                                  | **Logistic Regression**                                                                                   |
|---------------------------------|--------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|
| **Explanation**                | Predicts a continuous target variable $Y$ based on a linear relationship with features $X$.        | Predicts the probability of a binary outcome using the sigmoid function to constrain predictions between 0 and 1. |
| **Mathematical Representation**| $ \hat{Y} = \beta_0 + \sum_{j=1}^{n} \beta_j X_j$ | $P(Y = 1) = \frac{1}{1 + e^{-z}}, \quad z = \beta_0 + \sum_{j=1}^{n} \beta_j X_j $                |
| **Loss Function with L2 Penalty** | $ J(\boldsymbol{\beta}) = \frac{1}{2m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2 + \frac{\lambda}{2m} \sum_{j=1}^{n} \beta_j^2 $ | $ J(\boldsymbol{\beta}) = - \frac{1}{m} \sum_{i=1}^{m} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right] + \frac{\lambda}{2m} \sum_{j=1}^{n} \beta_j^2 $ |
| **Gradient with L2 Penalty**    | $ \nabla_{\beta_j} = -\frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y}_i) X_{ij} + \frac{\lambda}{m} \beta_j $ | $ \nabla_{\beta_j} = -\frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y}_i) X_{ij} + \frac{\lambda}{m} \beta_j $ |

### Notes:
1. **Loss Function Difference:** Linear regression uses Mean Squared Error (MSE) as the loss, while logistic regression uses Negative Log-Likelihood.
2. **Gradient Formula Similarity:** Both models update their parameters using gradient descent, with the addition of the L2 penalty $ \frac{\lambda}{m} \beta_j $.
3. **Sigmoid Function in Logistic Regression:** This function transforms the linear combination of predictors into a probability constrained between 0 and 1, making logistic regression suitable for binary outcomes.​⬤

# Evaluation Metrics

#### Summary:

* Precision: Focuses on the proportion of relevant instances (true positives) among the retrieved instances.
* Recall: Measures the proportion of actual positives correctly identified.
* F1 Score: A balance between precision and recall.
* Confusion Matrix: Provides a comprehensive view of the model’s performance.
* ROC Curve and AUC: Useful for evaluating the trade-off between recall and false positives at different thresholds, with AUC providing a summary measure.



#### Precision

Precision measures the proportion of **true positives predictions** out of all positive prediction made

$$ \text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$$

It is important to look at precision when the cost of false positives is high (eg
spam detection).

In [7]:
def precision_score(y_true, y_pred):

    true_positive = np.sum((y_true==1) & (y_pred==1))
    false_positive = np.sum((y_true==0) & (y_pred==1) )

    return true_positive/false_positive

#### Recall

Recall measures the proportion of **true positive predictions** out of all 
actual positive cases. 

$$ \text{Recal} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$$

It is import to look at recall when it is critical to capture as many positives as possible, such as in disease detection (missing an actual positive case is 
dangerous)

In [8]:
def recall_score(y_true, y_pred):

    true_positive = np.sum( (y_true == 1) & (y_pred == 1) )
    false_negative = np.sum( (y_true == 1) & (y_pred == 0) )

    return true_positive/(true_positive + false_negative)

#### F1 Score
The F1 Score is the **harmonic mean** of precision and recall. 

$$ \text{F1 Score} = 2\frac{\text{Precision} * \text{Recall}}{\text{Precision} + \text{Recall}}  $$

The metric is usefull when we need to balance the trade-off between precision and recall

#### ROC Curve and AUC

The ROC curve is a plot that show the trade-off between **true positive rate (recall)** and **false positive rate (FPR)**

THE AUC (Area Under the Curve) represents the area under the ROC curve and is a single number that summarizes the model's ability to distinguish classess: the higher the AUC, the better the model.